In [14]:
from bs4 import BeautifulSoup
import bs4
import re
import time
import operator
import requests
import logging
from pprint import pprint
import sys
def Crawl_allpost(boardName,startIndex,endIndex,keyword):
    all_post = open('all_article.txt','w',encoding='utf-8')
    #keyword exist flag
    exist = False
    for i in range(int(startIndex),int(endIndex)+1):
        page_url = '/bbs/'+boardName+'/index'+str(i)+'.html'
        prefix = 'https://www.ptt.cc'
        url = prefix + page_url
        r = requests.get(url,cookies={'over18': '1'},verify=True)
        soup = BeautifulSoup(r.text,'html.parser')
        all_title = soup.find_all('div','r-ent')
        for t in all_title:
            try:
                href = t.find('a')['href']
            except:
                continue
            
            title = t.find('a').text
            if(len(t.select('span.__cf_email__'))!=0):
                #print(title)
                filter_pat='\[email[\s]protected\]'
                pos = re.search(filter_pat,title).span()
                #print(pos)
                email_addr = decode_email(t.select('span.__cf_email__')[0]['data-cfemail'])
                #print(title[:pos[0]]+email_addr)
                title = title[:pos[0]]+email_addr
            #避免被誤為攻擊 每次至少間隔0.1秒
            time.sleep(0.01)
            #抓內文
            r = requests.get(prefix+href,cookies={'over18': '1'},verify=True)
            soup = BeautifulSoup(r.text,'html.parser')
            main_content = soup.find(id="main-content")
            filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'◆'] and v[:2] not in [u'--'] ]
            content = ""
            pushContent = ""
            #判斷內文開始
            contentFlag = False
            #判斷推文開始
            pushContentFlag = False
            a = 10
            if keyword in title:
                exist = True
            for text in filtered:
                if text == "":
                    continue
                elif text[0:2] == "時間":
                    contentFlag = True
                    continue
                elif text[0:6] == "※ 發信站:":
                    continue
                elif text[0:8] == "※ 文章網址:":
                    pushContentFlag = True
                    contentFlag = False
                    content = ''.join(content.split(',')[2:])
                    continue
                if contentFlag:
                    content = content + ',' + text
                    content = content.replace("\n", "")
                if pushContentFlag:
                    if text=="推" or text=="噓" or text=='→':
                        a=0
                    if a==2:
                        pushContent = pushContent + '\t' + text[2:].replace("\n","")
                    a = a + 1
                #判斷有無關鍵字
                if keyword in text:
                    exist = True
            if exist:
                #print(title)
                all_post.write(title+","+content+pushContent+"\n")
                exist = False
            time.sleep(0.01)

    all_post.close()


'''

command = sys.argv[1]

if command == 'crawl':
    boardName = sys.argv[2]
    startIndex = sys.argv[3]
    endIndex = sys.argv[4]
    try:
        keyword = sys.argv[5]
    except:
        keyword = ""
    #print(sys.argv)
    Crawl_allpost(boardName,startIndex,endIndex,keyword)
    '''

'\n\ncommand = sys.argv[1]\n\nif command == \'crawl\':\n    boardName = sys.argv[2]\n    startIndex = sys.argv[3]\n    endIndex = sys.argv[4]\n    try:\n        keyword = sys.argv[5]\n    except:\n        keyword = ""\n    #print(sys.argv)\n    Crawl_allpost(boardName,startIndex,endIndex,keyword)\n    '

In [15]:
Crawl_allpost('Gossiping', '1', '3', '' )

In [2]:
boardName = 'Beauty'
i = 2000;
page_url = '/bbs/'+boardName+'/index'+str(i)+'.html'
prefix = 'https://www.ptt.cc'
url = prefix + page_url
r = requests.get(url,cookies={'over18': '1'},verify=True)
soup = BeautifulSoup(r.text,'html.parser')
print(soup)

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>看板 Beauty 文章列表 - 批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.25/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.25/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.25/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.25/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.25/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Beauty/index.html"><span class="board-label">看板 </span>Beauty</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>
<div 

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
import operator
import sys
r = requests.get('https://www.ptt.cc/bbs/Gossiping/M.1528989757.A.E93.html',cookies={'over18': '1'},verify=True)
soup = BeautifulSoup(r.text,'html.parser')
main_content = soup.find(id="main-content")
filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'◆'] and v[:2] not in [u'--'] ]
#content = soup.find('div' , {'class': 'bbs-screen bbs-content'})
#content_line = content.text.split("\n")
content = ""
pushContent = ""
contentFlag = False
pushContentFlag = False
a = 0
for text in filtered :
    if text == "":
        continue
    elif text[0:2] == "時間":
        contentFlag = True
        continue
    elif text[0:6] == "※ 發信站:":
        continue
    elif text[0:8] == "※ 文章網址:":
        pushContentFlag = True
        contentFlag = False
        content = content.split(',')[2:]
        continue
    if contentFlag:
        content = content + ',' + text
    if pushContentFlag:
        if a%4==3:
            pushContent = pushContent + '\t' + text[2:]
        a = a + 1
    #if keyword in text:
    #    exist = 1
print(content+ pushContent)

TypeError: can only concatenate list (not "str") to list